In [1]:
from selenium import webdriver  # 動態爬蟲套件
from bs4 import BeautifulSoup   # 原始碼解析器

import re
import pandas as pd
import numpy as np

## 巴哈爬蟲

In [2]:
%%time

driver = webdriver.Chrome('C:\ChromeDrive\chromedriver.exe') # Chrome Driver的路徑
driver.get('https://forum.gamer.com.tw/?c=94')               # 欲爬網址 (熱門手機遊戲)


data = []
url =[]

for i in range(1, 279):  
    
    driver.find_element_by_link_text('清單').click()                          # 先點'清單 '
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    for ele in soup.select('.BH-table'):                                     # 指定元素:巴哈中間的內容，爬完存到 data裡
        data.append(ele.text)
    
    for ele in soup.find_all('table', class_='BH-table BH-table1'):          # 指定元素內所有的超連結，爬完存到 url裡
        for a in ele.find_all('a'):
            url.append(a.get('href'))

    driver.find_element_by_class_name('next').click()                        # 點下一頁

    
driver.close()                                              #關閉 Chrome Driver

Wall time: 4min 9s


## 資料整理

In [3]:
# 處理 data list


# 將遊戲排名換為自訂標記，並依照標記將各遊戲資訊分開

str1 = ''.join(data)
list1 = re.sub('  \d ',' xxxxx ',str1)
list1 = re.sub('  \d\d ',' xxxxx ',list1)
list1 = re.sub('  \d\d\d ',' xxxxx ',list1)
list1 = re.sub('  \d\d\d\d ',' xxxxx ',list1)
list1 = list1.split(' xxxxx ')

A = [list1[i:i+1] for i in range(0,len(list1),1)]


# 將各遊戲資訊存成 list

Title=[]
for i in A:
    str2=''.join(i)
    B = str2[str2.find(' xxxxx ')+1 : str2.find('  ')]
    Title.append(B)
Title.remove('')
    
Popular_articles = []
for i in A:
    str2=''.join(i)
    C = str2[str2.find('  ') : str2.find('板主')-2]
    Popular_articles.append(C)
Popular_articles.remove('')
    
Popularity=[]
for i in A:
    str2=''.join(i)
    D = str2[str2.find('昨日人氣：')+5 : str2.find('昨日文章')-2]
    Popularity.append(D)
Popularity.remove('')
    
Articles = []
for i in A:
    str2=''.join(i)
    E = str2[str2.find('昨日文章：')+5 :]
    Articles.append(E)
Articles.remove('')


In [4]:
# 處理 url list

del url[0]
del url[1::2]
url = ['https://forum.gamer.com.tw/'+i for i in url]

In [5]:
df1 = pd.DataFrame(Title,columns = ['遊戲名稱'])
df2 = pd.DataFrame(Popular_articles,columns = ['熱門文章'])
df3 = pd.DataFrame(url,columns = ['熱門文章連結'])
df4 = pd.DataFrame(Popularity,columns = ['昨日人氣'])
df5 = pd.DataFrame(Articles,columns = ['昨日文章數'])

df = pd.concat([df1, df2, df3, df4, df5], axis=1, sort=False)

In [7]:
df.tail(5)

,遊戲名稱,熱門文章,熱門文章連結,昨日人氣,昨日文章數
8335,100 日公主◆美男宮殿現代樂章,-------,https://forum.gamer.com.tw/A.php?bsn=31648,1,0
8336,英雄樂園,-------,https://forum.gamer.com.tw/A.php?bsn=35526,0,0
8337,狐姬零,-------,https://forum.gamer.com.tw/A.php?bsn=35523,0,0
8338,刷牙勇者,-------,https://forum.gamer.com.tw/A.php?bsn=35525,0,0
8339,Triple Agent 三面間諜,-------,https://forum.gamer.com.tw/A.php?bsn=35520,0,0


In [8]:
df.index = df.index + 1
df.to_csv('巴哈姆特手遊排名_20181102.csv', encoding='utf_8_sig')